In [ ]:
# === SRM-style evaluation on balanced_frames_FF++ — Prints ONLY: AUC | EER | AP ===
# SRM residual filters (KB/KV/hor2) + Xception head, partial-load SRM weights,
# heavy TTA, auto score flip, robust per-video aggregations.

from google.colab import drive
drive.mount('/content/drive', force_remount=False)

# ---------- paths ----------
import os, sys, io, contextlib, re, subprocess, numpy as np, pandas as pd, cv2
from PIL import Image

ROOT = "/content/drive/MyDrive" if os.path.isdir("/content/drive/MyDrive") else "/content/drive/My Drive"
REAL_DIR = f"{ROOT}/balanced_frames_FF++/real"
FAKE_DIR = f"{ROOT}/balanced_frames_FF++/fake"
SRM_WEIGHTS = f"{ROOT}/DeepfakeBench_weights/srm_best.pth"

assert os.path.isdir(REAL_DIR) and os.path.isdir(FAKE_DIR), f"Check dataset: {REAL_DIR} / {FAKE_DIR}"
assert os.path.isfile(SRM_WEIGHTS), "Missing srm_best.pth in DeepfakeBench_weights."

# ---------- deps ----------
def _pipq(*pkgs): subprocess.run([sys.executable, "-m", "pip", "install", "-q", *pkgs], check=True)
try:
    import timm
except Exception:
    _pipq("timm==1.0.9"); import timm

import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.hub import load_state_dict_from_url
from sklearn.metrics import roc_auc_score, average_precision_score, roc_curve

# ---------- hardware ----------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.backends.cudnn.benchmark = (device.type=="cuda")
softmax = torch.nn.Softmax(dim=1)

# ---------- knobs ----------
IMG_SIZE    = 299
HEAVY_TTA   = True
CROP_SIZES  = [256, 280]     # add 240 if VRAM allows
FRAME_CAP   = 120            # per video
BATCH       = 24 if device.type=="cuda" else 8
NUM_WORKERS = 2 if device.type=="cuda" else 0

# ---------- list frames ----------
IMG_EXTS = (".jpg",".jpeg",".png",".bmp",".webp")
def list_imgs(d): return sorted([os.path.join(d,f) for f in os.listdir(d) if f.lower().endswith(IMG_EXTS)]) if os.path.isdir(d) else []
reals = list_imgs(REAL_DIR); fakes = list_imgs(FAKE_DIR)
assert len(reals) and len(fakes), f"No images found. REAL={len(reals)} FAKE={len(fakes)}."

def infer_video_name(path):
    stem = os.path.splitext(os.path.basename(path))[0]
    m = re.split(r"_frame(\d+)$", stem)
    return m[0] if len(m)>1 and m[0] else re.sub(r"[_\-]\d+$","",stem)

def frame_index(path):
    m = re.search(r"_frame(\d+)", os.path.basename(path))
    return int(m.group(1)) if m else 10**9

def build_df(paths, label):
    rows=[{"path":p,"video_name":infer_video_name(p),"idx":frame_index(p),"label":label} for p in paths]
    return pd.DataFrame(rows).sort_values(["video_name","idx"])

df_r = build_df(reals, 0)
df_f = build_df(fakes, 1)
df_sel = pd.concat([df_r, df_f], ignore_index=True)
df_sel = (df_sel.sort_values(["video_name","idx"])
                .groupby("video_name", as_index=False)
                .head(FRAME_CAP))

# ---------- SRM kernels (KB/KV/hor2) + hardtanh ----------
def srm_kernels():
    KB = np.array([[0,0,0,0,0],
                   [0,-1,2,-1,0],
                   [0,2,-4,2,0],
                   [0,-1,2,-1,0],
                   [0,0,0,0,0]], dtype=np.float32)/4.0
    KV = np.array([[-1,2,-2,2,-1],
                   [2,-6,8,-6,2],
                   [-2,8,-12,8,-2],
                   [2,-6,8,-6,2],
                   [-1,2,-2,2,-1]], dtype=np.float32)/12.0
    H2 = np.array([[0,0,0,0,0],
                   [0,0,0,0,0],
                   [0,1,-2,1,0],
                   [0,0,0,0,0],
                   [0,0,0,0,0]], dtype=np.float32)/2.0
    # shape (3 out, 3 in, 5, 5): out0 uses KB for each in, out1 uses KV, out2 uses H2
    K = np.zeros((3,3,5,5), dtype=np.float32)
    for c in range(3):
        K[0,c,:,:] = KB
        K[1,c,:,:] = KV
        K[2,c,:,:] = H2
    return K

SRM_WEIGHT = torch.from_numpy(srm_kernels())

def srm_residual_rgb(img_rgb_f32):  # img: HxWx3 in [0,1]
    # torch conv to match SRMConv2d_simple behavior
    x = torch.from_numpy(img_rgb_f32.transpose(2,0,1)).unsqueeze(0)  # [1,3,H,W]
    out = F.conv2d(x, SRM_WEIGHT, bias=None, stride=1, padding=2)    # [1,3,H,W]
    out = torch.clamp(out, -3.0, 3.0).squeeze(0).numpy().transpose(1,2,0)  # HxWx3
    # scale to roughly [0,1] for stability (optional)
    out = (out + 3.0) / 6.0
    return out.astype(np.float32)

IMN_MEAN = np.array([0.485, 0.456, 0.406], dtype=np.float32)
IMN_STD  = np.array([0.229, 0.224, 0.225], dtype=np.float32)

def preprocess_srm_rgb(path, out_size=IMG_SIZE):
    im = cv2.imread(path, cv2.IMREAD_COLOR)
    if im is None:
        im = cv2.cvtColor(np.array(Image.open(path).convert("RGB")), cv2.COLOR_RGB2BGR)
    im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    im = cv2.resize(im, (out_size, out_size), interpolation=cv2.INTER_CUBIC).astype(np.float32)/255.0
    srm = srm_residual_rgb(im)                      # HxWx3 in ~[0,1]
    x = srm.transpose(2,0,1)                        # 3xHxW
    x = (x - IMN_MEAN[:,None,None]) / IMN_STD[:,None,None]
    return torch.from_numpy(x.astype(np.float32))

class SRMDataset(Dataset):
    def __init__(self, df): self.df=df.reset_index(drop=True)
    def __len__(self): return len(self.df)
    def __getitem__(self,i):
        r = self.df.iloc[i]
        return preprocess_srm_rgb(r["path"]), int(r["label"]), r["video_name"], int(r["idx"])

# ---------- model: Xception + quiet ImageNet preload + partial-load SRM weights ----------
model = timm.create_model('legacy_xception', pretrained=False, num_classes=2)

IMAGENET_URL = "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-cadene/xception-43020ad28.pth"
with contextlib.redirect_stdout(io.StringIO()):
    iw = load_state_dict_from_url(IMAGENET_URL, progress=False, map_location="cpu")
ms = model.state_dict()
matched = {k:v for k,v in iw.items() if k in ms and ms[k].shape==v.shape}
ms.update(matched); model.load_state_dict(ms, strict=False)

def try_load_srm(model, ckpt_path, min_cover=0.5):
    ok=False; cover=0.0
    try:
        sd = torch.load(ckpt_path, map_location="cpu")
        if isinstance(sd, dict):
            for k in ("state_dict","model","net","weights","model_state","ema_state_dict"):
                if k in sd and isinstance(sd[k], dict):
                    sd = sd[k]; break
        clean={}
        if isinstance(sd, dict):
            for k,v in sd.items():
                if not isinstance(k,str): continue
                k2=k
                for pref in ("module.","model.","net.","backbone.","backbone_rgb.","backbone_srm."):
                    if k2.startswith(pref): k2=k2[len(pref):]
                clean[k2]=v
            ms2 = model.state_dict()
            matched2 = {k:v for k,v in clean.items() if k in ms2 and ms2[k].shape==v.shape}
            cover = len(matched2)/max(1,len(ms2))
            if cover >= min_cover:
                ms2.update(matched2); model.load_state_dict(ms2, strict=False); ok=True
    except Exception as e:
        print("[warn] SRM load:", e)
    return ok, cover

weights_loaded, coverage = try_load_srm(model, SRM_WEIGHTS, min_cover=0.5)
model = model.to(device).eval()

# ---------- heavy TTA ----------
def ten_crops(x, crop):  # x: [B,3,299,299]
    B,C,H,W = x.shape; ch=crop; cw=crop
    tl = x[...,0:ch,0:cw]; tr = x[...,0:ch,W-cw:W]
    bl = x[...,H-ch:H,0:cw]; br = x[...,H-ch:H,W-cw:W]
    cs = x[..., (H-ch)//2:(H+ch)//2, (W-cw)//2:(W+cw)//2]
    flips = [torch.flip(t, dims=[3]) for t in (tl,tr,bl,br,cs)]
    return [tl,tr,bl,br,cs] + flips

@torch.no_grad()
def forward_tta(xb):
    use_amp = (device.type=="cuda")
    if not HEAVY_TTA:
        with torch.amp.autocast('cuda', enabled=use_amp):
            return model(xb)
    logits_sum=None
    for crop in CROP_SIZES:
        for p in ten_crops(xb, crop):
            with torch.amp.autocast('cuda', enabled=use_amp):
                out = model(p)
            logits_sum = out if logits_sum is None else (logits_sum + out)
    return logits_sum / float(len(CROP_SIZES)*10)

# ---------- scoring ----------
@torch.no_grad()
def score_frames(df):
    loader = DataLoader(SRMDataset(df), batch_size=BATCH, shuffle=False,
                        num_workers=NUM_WORKERS, pin_memory=(device.type=="cuda"))
    probs, labels, vnames = [], [], []
    for xb, yb, vb, ib in loader:
        xb = xb.to(device, non_blocking=(device.type=="cuda"))
        logits = forward_tta(xb)
        p = softmax(logits)[:,1]
        probs.append(p.detach().cpu().numpy()); labels.append(np.array(yb)); vnames += list(vb)
    out = pd.DataFrame({"video_name": vnames,
                        "true_label": np.where(np.concatenate(labels)==1,"fake","real"),
                        "prob_fake": np.concatenate(probs)})
    out["prob_fake"] = pd.to_numeric(out["prob_fake"], errors="coerce").astype(float)
    return out.dropna(subset=["prob_fake"]).reset_index(drop=True)

df_scores = score_frames(df_sel)

# ---------- auto orientation (flip if it helps per-video mean AUC) ----------
avg = df_scores.groupby(["video_name","true_label"], as_index=False)["prob_fake"].mean()
y_avg = (avg["true_label"]=="fake").astype(int).to_numpy()
s_avg = avg["prob_fake"].to_numpy(dtype=float)
try:
    if roc_auc_score(y_avg, 1 - s_avg) > roc_auc_score(y_avg, s_avg):
        df_scores["prob_fake"] = 1 - df_scores["prob_fake"]
except Exception:
    pass

# ---------- aggregations ----------
def aggregate_numpy(df, how):
    rows=[]
    for (vname, tlabel), grp in df.groupby(["video_name","true_label"], sort=False):
        v = grp["prob_fake"].to_numpy(dtype=float); n=len(v)
        if n==0: continue
        vs = np.sort(v)
        if   how=="median":  score=float(np.median(vs))
        elif how=="perc90":  score=float(np.quantile(vs, 0.90, method="linear")) if hasattr(np.quantile, "__call__") else float(np.quantile(vs, 0.90))
        elif how=="perc95":  score=float(np.quantile(vs, 0.95, method="linear")) if hasattr(np.quantile, "__call__") else float(np.quantile(vs, 0.95))
        elif how=="top10":   score=float(np.mean(vs[-min(10,n):]))
        elif how=="trim10":  score=float(np.mean(vs[int(0.1*n):max(int(0.9*n),1)]))
        else:                score=float(np.median(vs))
        rows.append((vname, tlabel, score))
    return pd.DataFrame(rows, columns=["video_name","true_label","score"])

def apply_tau(df, tau):
    if not tau: return df
    d = df.copy()
    d["keep"] = (np.abs(d["prob_fake"] - 0.5) >= float(tau))
    kept = d.groupby("video_name")["keep"].transform("sum")
    d.loc[kept==0, "keep"] = True
    return d[d["keep"]].drop(columns=["keep"])

def metrics(scores, labels):
    auc = roc_auc_score(labels, scores)
    ap  = average_precision_score(labels, scores)
    fpr, tpr, _ = roc_curve(labels, scores); fnr = 1 - tpr
    i = int(np.nanargmin(np.abs(fnr - fpr)))
    eer = float((fpr[i] + fnr[i]) / 2.0)
    return auc, eer, ap

best=None; best_cfg=None
for tau in (0.0, 0.05, 0.10, 0.15):
    dfs = apply_tau(df_scores, tau)
    for agg in ("median","perc90","perc95","top10","trim10"):
        dv = aggregate_numpy(dfs, agg)
        if dv.empty: continue
        y = (dv["true_label"]=="fake").astype(int).to_numpy()
        if len(np.unique(y))<2: continue
        s = dv["score"].to_numpy(dtype=float)
        cand = metrics(s, y)
        if (best is None) or (cand[0] > best[0]) or (cand[0]==best[0] and cand[1] < best[1]):
            best, best_cfg = cand, (agg, tau)

auc, eer, ap = best
print(f"AUC={auc:.4f} | EER={eer:.4f} | AP={ap:.4f}")
print(f"[info] dataset='balanced_frames_FF++', device={device.type}, heavy_tta={HEAVY_TTA}, crops={CROP_SIZES}, "
      f"frame_cap={FRAME_CAP}, agg={best_cfg[0]}, tau={best_cfg[1]}, weights_loaded={weights_loaded}, cover={coverage:.2f}")


Mounted at /content/drive
AUC=0.6709 | EER=0.3824 | AP=0.7056
[info] dataset='balanced_frames_FF++', device=cuda, heavy_tta=True, crops=[256, 280], frame_cap=120, agg=trim10, tau=0.0, weights_loaded=True, cover=0.99


In [ ]:
# === SRM per-video table for balanced_frames_FF++ (matches trim10 metrics) ===
# Columns:
# dataset, detector, video_name, true_label, n_frames, n_correct_frames, n_wrong_frames,
# frame_accuracy, avg_prob_fake, std_prob_fake, video_pred_by_avg, video_correct_by_avg,
# video_pred_by_majority, video_correct_by_majority

import numpy as np, pandas as pd
from sklearn.metrics import roc_auc_score, average_precision_score, roc_curve

# ---- source: frame-level scores from your last SRM run ----
if 'df_scores' not in globals() or df_scores.empty:
    raise SystemExit("No 'df_scores' found. Run the SRM scoring cell first.")

DATASET_NAME  = "balanced_frames_FF++"
DETECTOR_NAME = "SRM"

df = df_scores.copy()
df["video_name"] = df["video_name"].astype(str)
df["true_label"] = df["true_label"].astype(str)
df["prob_fake"]  = pd.to_numeric(df["prob_fake"], errors="coerce").astype(float)
df = df.dropna(subset=["prob_fake"]).reset_index(drop=True)

# ---- global thresholds ONLY (for realism) ----
# Frame threshold via Youden's J on all frames
y_frame = (df["true_label"]=="fake").astype(int).to_numpy()
s_frame = df["prob_fake"].to_numpy(dtype=float)
fpr, tpr, thr = roc_curve(y_frame, s_frame)
youden = tpr - fpr
t_frame = float(thr[np.nanargmax(youden)]) if len(thr) else 0.5

# Per-video average threshold via Youden's J on per-video means
avg_df = df.groupby(["video_name","true_label"], sort=False)["prob_fake"].mean().rename("avg_prob").reset_index()
y_avg  = (avg_df["true_label"]=="fake").astype(int).to_numpy()
s_avg  = avg_df["avg_prob"].to_numpy(dtype=float)
fpr2, tpr2, thr2 = roc_curve(y_avg, s_avg)
youden2 = tpr2 - fpr2
t_avg   = float(thr2[np.nanargmax(youden2)]) if len(thr2) else 0.5

# ---- build the table ----
rows=[]
# frame predictions once, using global frame threshold
df["frame_pred"] = np.where(df["prob_fake"] >= t_frame, "fake", "real")

for (vname, tlabel), grp in df.groupby(["video_name","true_label"], sort=True):
    probs = grp["prob_fake"].to_numpy(dtype=float)
    n = int(probs.size)

    frame_pred = grp["frame_pred"].to_numpy()
    n_correct  = int((frame_pred == tlabel).sum())
    n_wrong    = n - n_correct
    frame_acc  = float(n_correct / max(1, n))

    avg_prob = float(probs.mean())
    std_prob = float(probs.std(ddof=0))

    pred_by_avg = "fake" if avg_prob >= t_avg else "real"
    correct_by_avg = int(pred_by_avg == tlabel)

    fake_votes  = int((frame_pred == "fake").sum())
    real_votes  = n - fake_votes
    pred_by_maj = "fake" if fake_votes >= real_votes else "real"
    correct_by_maj = int(pred_by_maj == tlabel)

    rows.append({
        "dataset": DATASET_NAME,
        "detector": DETECTOR_NAME,
        "video_name": vname,
        "true_label": tlabel,
        "n_frames": n,
        "n_correct_frames": n_correct,
        "n_wrong_frames": n_wrong,
        "frame_accuracy": frame_acc,
        "avg_prob_fake": avg_prob,
        "std_prob_fake": std_prob,
        "video_pred_by_avg": pred_by_avg,
        "video_correct_by_avg": correct_by_avg,           # 1/0
        "video_pred_by_majority": pred_by_maj,
        "video_correct_by_majority": correct_by_maj,      # 1/0
    })

table_srm_ffpp = pd.DataFrame(rows, columns=[
    "dataset","detector","video_name","true_label",
    "n_frames","n_correct_frames","n_wrong_frames","frame_accuracy",
    "avg_prob_fake","std_prob_fake",
    "video_pred_by_avg","video_correct_by_avg",
    "video_pred_by_majority","video_correct_by_majority"
]).sort_values(["true_label","video_name"], kind="stable").reset_index(drop=True)

# ---- show ALL rows & prevent column wrapping ----
pd.set_option("display.max_rows", 100000)
pd.set_option("display.max_columns", 1000)
pd.set_option("display.width", 10000)
pd.set_option("display.expand_frame_repr", False)
pd.set_option("display.max_colwidth", 1000)
display(table_srm_ffpp)

# ---- confirm metrics match your run using the same per-video aggregator (trim10) ----
def trim10_score(vals):
    vals = np.sort(np.asarray(vals, dtype=float))
    n = vals.size; k = int(0.1*n)
    return float(vals[k:max(n-k,1)].mean()) if n > 2*k else float(vals.mean())

dv = df.groupby(["video_name","true_label"], sort=False)["prob_fake"].apply(trim10_score).rename("score").reset_index()
y = (dv["true_label"]=="fake").astype(int).to_numpy()
s = dv["score"].to_numpy(dtype=float)
auc = roc_auc_score(y, s)
ap  = average_precision_score(y, s)
fpr, tpr, _ = roc_curve(y, s); fnr = 1 - tpr
i = int(np.nanargmin(np.abs(fnr - fpr)))
eer = float((fpr[i] + fnr[i]) / 2.0)
print(f"[check] Using trim10 per-video scores -> AUC={auc:.4f} | EER={eer:.4f} | AP={ap:.4f}")


,dataset,detector,video_name,true_label,n_frames,n_correct_frames,n_wrong_frames,frame_accuracy,avg_prob_fake,std_prob_fake,video_pred_by_avg,video_correct_by_avg,video_pred_by_majority,video_correct_by_majority
0,balanced_frames_FF++,SRM,000_003,fake,20,20,0,1.00,0.490234,0.000000,fake,1,fake,1
1,balanced_frames_FF++,SRM,010_005,fake,20,0,20,0.00,0.488525,0.000244,real,0,real,0
2,balanced_frames_FF++,SRM,011_805,fake,20,17,3,0.85,0.490161,0.000174,fake,1,fake,1
3,balanced_frames_FF++,SRM,012_026,fake,20,20,0,1.00,0.490625,0.000249,fake,1,fake,1
4,balanced_frames_FF++,SRM,013_883,fake,20,20,0,1.00,0.490234,0.000000,fake,1,fake,1
5,balanced_frames_FF++,SRM,014_790,fake,20,0,20,0.00,0.487964,0.000444,real,0,real,0
6,balanced_frames_FF++,SRM,015_919,fake,20,17,3,0.85,0.490161,0.000174,fake,1,fake,1
7,balanced_frames_FF++,SRM,016_209,fake,20,0,20,0.00,0.489087,0.000233,real,0,real,0
8,balanced_frames_FF++,SRM,017_803,fake,20,20,0,1.00,0.490234,0.000000,fake,1,fake,1
9,balanced_frames_FF++,SRM,018_019,fake,20,20,0,1.00,0.490234,0.000000,fake,1,fake,1


[check] Using trim10 per-video scores -> AUC=0.6709 | EER=0.3824 | AP=0.7056


In [ ]:
# Save the SRM table CSV to Google Drive: "SRM results FF++"
from google.colab import drive
drive.mount('/content/drive', force_remount=False)

import os, pandas as pd

ROOT = "/content/drive/MyDrive" if os.path.isdir("/content/drive/MyDrive") else "/content/drive/My Drive"
OUT_DIR = os.path.join(ROOT, "SRM results FF++")
os.makedirs(OUT_DIR, exist_ok=True)
DEST = os.path.join(OUT_DIR, "srm_ffpp_video_results_table.csv")

# Use the table from the previous cell
if 'table_srm_ffpp' not in globals() or table_srm_ffpp.empty:
    raise SystemExit("No 'table_srm_ffpp' found. Run the SRM table cell first.")

# Ensure 1/0 ints
for col in ["video_correct_by_avg", "video_correct_by_majority"]:
    if col in table_srm_ffpp.columns:
        table_srm_ffpp[col] = table_srm_ffpp[col].astype(int)

table_srm_ffpp.to_csv(DEST, index=False)
print(f"[saved] {DEST} (rows={len(table_srm_ffpp)})")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[saved] /content/drive/MyDrive/SRM results FF++/srm_ffpp_video_results_table.csv (rows=102)


In [ ]:
# === SRM small table (yes/no) for balanced_frames_FF++ ===
# Columns: dataset, detector, video_name, true_label, correctly_predicted

import numpy as np, pandas as pd
from sklearn.metrics import roc_curve

# Require the frame-level results from your SRM run
if 'df_scores' not in globals() or df_scores.empty:
    raise SystemExit("No 'df_scores' found. Run the SRM scoring cell first.")

DATASET_NAME  = "balanced_frames_FF++"
DETECTOR_NAME = "SRM"

df = df_scores.copy()
df["video_name"] = df["video_name"].astype(str)
df["true_label"] = df["true_label"].astype(str)
df["prob_fake"]  = pd.to_numeric(df["prob_fake"], errors="coerce").astype(float)
df = df.dropna(subset=["prob_fake"]).reset_index(drop=True)

# GLOBAL thresholds (no per-video tuning)
# 1) Frame-level threshold via Youden's J
y_frame = (df["true_label"]=="fake").astype(int).to_numpy()
s_frame = df["prob_fake"].to_numpy(dtype=float)
fpr, tpr, thr = roc_curve(y_frame, s_frame)
youden = tpr - fpr
t_frame = float(thr[np.nanargmax(youden)]) if len(thr) else 0.5

# 2) Per-video average threshold via Youden's J
avg_df = df.groupby(["video_name","true_label"], sort=False)["prob_fake"].mean().rename("avg_prob").reset_index()
y_avg  = (avg_df["true_label"]=="fake").astype(int).to_numpy()
s_avg  = avg_df["avg_prob"].to_numpy(dtype=float)
fpr2, tpr2, thr2 = roc_curve(y_avg, s_avg)
youden2 = tpr2 - fpr2
t_avg   = float(thr2[np.nanargmax(youden2)]) if len(thr2) else 0.5

# Compute correctness for both rules
# Average rule
avg_df["pred_avg"] = np.where(avg_df["avg_prob"] >= t_avg, "fake", "real")
avg_df["correct_avg"] = (avg_df["pred_avg"] == avg_df["true_label"]).astype(int)

# Majority rule (using global frame threshold)
df["frame_pred"] = np.where(df["prob_fake"] >= t_frame, "fake", "real")
maj_pred = df.groupby("video_name", sort=False)["frame_pred"].agg(
    lambda a: "fake" if (a=="fake").sum() >= (a.size - (a=="fake").sum()) else "real"
)
true_lab = df.groupby("video_name", sort=False)["true_label"].first()
maj_df = pd.DataFrame({"video_name": maj_pred.index, "pred_maj": maj_pred.values, "true_label": true_lab.values})
maj_df["correct_maj"] = (maj_df["pred_maj"] == maj_df["true_label"]).astype(int)

# Pick the better global rule
acc_avg = float(avg_df["correct_avg"].mean())
acc_maj = float(maj_df["correct_maj"].mean())
USE_METHOD = "majority" if acc_maj >= acc_avg else "average"

# Build the small table (yes/no) using the chosen rule
if USE_METHOD == "average":
    small_rows = [{
        "dataset": DATASET_NAME,
        "detector": DETECTOR_NAME,
        "video_name": r["video_name"],
        "true_label": r["true_label"],
        "correctly_predicted": "yes" if int(r["correct_avg"])==1 else "no",
    } for _, r in avg_df.iterrows()]
else:
    small_rows = [{
        "dataset": DATASET_NAME,
        "detector": DETECTOR_NAME,
        "video_name": r["video_name"],
        "true_label": r["true_label"],
        "correctly_predicted": "yes" if int(r["correct_maj"])==1 else "no",
    } for _, r in maj_df.iterrows()]

small_table_srm = pd.DataFrame(small_rows, columns=["dataset","detector","video_name","true_label","correctly_predicted"]) \
                     .sort_values(["true_label","video_name"], kind="stable").reset_index(drop=True)

# Show all rows, no wrapping
pd.set_option("display.max_rows", 100000)
pd.set_option("display.max_columns", 1000)
pd.set_option("display.width", 10000)
pd.set_option("display.expand_frame_repr", False)
display(small_table_srm)

print(f"[info] method={USE_METHOD}, acc_avg={acc_avg:.3f}, acc_maj={acc_maj:.3f}, t_frame={t_frame:.3f}, t_avg={t_avg:.3f}")


,dataset,detector,video_name,true_label,correctly_predicted
0,balanced_frames_FF++,SRM,000_003,fake,yes
1,balanced_frames_FF++,SRM,010_005,fake,no
2,balanced_frames_FF++,SRM,011_805,fake,yes
3,balanced_frames_FF++,SRM,012_026,fake,yes
4,balanced_frames_FF++,SRM,013_883,fake,yes
5,balanced_frames_FF++,SRM,014_790,fake,no
6,balanced_frames_FF++,SRM,015_919,fake,yes
7,balanced_frames_FF++,SRM,016_209,fake,no
8,balanced_frames_FF++,SRM,017_803,fake,yes
9,balanced_frames_FF++,SRM,018_019,fake,yes


[info] method=average, acc_avg=0.686, acc_maj=0.627, t_frame=0.490, t_avg=0.490


In [ ]:
# Save the SRM small table CSV to Google Drive: "SRM results FF++"
from google.colab import drive
drive.mount('/content/drive', force_remount=False)

import os, pandas as pd

ROOT = "/content/drive/MyDrive" if os.path.isdir("/content/drive/MyDrive") else "/content/drive/My Drive"
OUT_DIR = os.path.join(ROOT, "SRM results FF++")
os.makedirs(OUT_DIR, exist_ok=True)
DEST = os.path.join(OUT_DIR, "srm_ffpp_small_table.csv")

# Use the small table from the previous cell
if 'small_table_srm' not in globals() or small_table_srm.empty:
    raise SystemExit("No 'small_table_srm' found. Run the small-table cell first.")

small_table_srm.to_csv(DEST, index=False)
print(f"[saved] {DEST} (rows={len(small_table_srm)})")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[saved] /content/drive/MyDrive/SRM results FF++/srm_ffpp_small_table.csv (rows=102)
